# Nixie-Clock Power Supply

## Requirements:

In [38]:
import math

class OutputWinding:
    """Power Supply Output Winding"""
    
    name:str
    volts:float
    load:float
    ocp:float #Over-current protection level in A
    lof:float #Load occupying factor
    N:int #Number of windings
    
    def __init__(self,name:str, vout:float, load:float):
        self.name = name
        self.volts = vout
        self.load = load
        self.lof = 0
        self.N = 0
        
    @property
    def power(self):
        return self.volts * self.load
    
    def __repr__(self):
        return f"{self.name} : [{self.volts:2.2f}V, {self.load:2.2f}A, {self.power:2.2f}W] : N={self.N} | lof={self.lof*100:2.2f}%"
    
class FlybackConverter:
    """ Flyback DC/DC Power converter """
    
    outputs:list[OutputWinding] = []  #List of Flyback Output Rails
    fsw:float  #Switching Frequency
    effmin: float #Minimum Efficiency Target
    dmax:float #Maximum Duty Cycle
    
    def __init__(self, vin_range:list, outputs:list[OutputWinding], fsw:float, dmax:float, effmin:float, ocp:float=0.650):
        self.vin_range = vin_range
        self.vinmin = min(self.vin_range)
        self.vinmax = max(self.vin_range)
        self.outputs = outputs
        self.effmin = effmin
        self.fsw = fsw
        self.dmax = dmax
        self.ocp = ocp
        self._refresh_lof()
        self.calc_fet_currents()
    
    def add_output_winding(self, output:OutputWinding):
        outputs.append(output)
        self._refresh_lof()
    
    def _refresh_lof(self):
        """ Refresh winding Load Occupying Factors"""
        for output in self.outputs:
            output.lof = output.power/self.total_output_power
    
    @property
    def total_output_power(self):
        pout_total = 0
        for output in self.outputs:
            pout_total += output.power
        return pout_total
    
    @property
    def input_power(self):
        return self.total_output_power/self.effmin
            
    def __repr__(self):
        s = f"Vin(min) = {self.vinmin:2.2f}VDC\n"
        s +=  f"Vin(max) = {self.vinmax:2.2f}VDC\n"
        s +=  f"Pin = {self.input_power:2.2f}W (effmin = {self.effmin*100:2.2f}%)"
        return s
    
    @property
    def vro(self):
        """ Reflected Output Voltage """
        
        return self.dmax/(1 - self.dmax) * self.vinmin
        
    @property
    def vds_nom(self):
        """ Returns ballpark worst case FET Vds voltage"""
        return self.vro + self.vinmax
       
    @property
    def lm_max(self):
        """ Max Primary Inductance to operate in DCM """
        return (self.vinmin * self.dmax)**2 / (2*self.input_power*self.fsw)
    
    def calc_fet_currents(self):
        """ Derives Primary FET peak and RMS currents"""
        try:
            lm = self.lm
        except AttributeError:
            lm = self.lm_max
        iedc = self.input_power/(self.vinmin*self.dmax)
        dilm = self.vinmin*self.dmax / (lm*self.fsw)
        self.ids_pk = iedc + dilm/2.0
        self.ids_rms = math.sqrt((3*iedc**2 + (dilm/2.0)**2)*self.dmax/3.0) 
        
        
    def np_min(self,bsat:float, ae:float):
        try:
            lm = self.lm
        except AttributeError:
            lm = self.lm_max
            
        return lm*self.ocp/(bsat*ae)
    
    def get_magcore_k_const(self, bmax:float, imax:float, coil_res:float, ku:float):
        """ Calculate magnetic core geometric constant Kg """
        
        res_cu = 1.72e-8 # Ohm-m
        
        try:
            lm = self.lm
        except AttributeError:
            lm = self.lm_max
            
        return lm**2*imax**2*res_cu/(bmax**2*coil_res*ku)
        
        
    
    
    
    
    
    
    
    

    
    
    
    

In [39]:
output1 = OutputWinding(name = 'nixie rail', vout = 180.0, load = 25e-3)
output2 = OutputWinding(name = 'logic rail', vout = 3.3, load = 280e-3)
flyback = FlybackConverter(vin_range = [115, 373], outputs=[output1,output2], fsw=67e3, dmax=0.45, effmin=0.75)


In [41]:
flyback.lm_max

0.0027634874232267863

# Core Selection:

In [36]:
bsat = 0.35 #T
ae = 52.2 #mm^2
ocp = 0.65
res = 1.0
ku = 0.3

np_min = flyback.lm_max * ocp / (bsat*ae) * 10**6

print(round(np_min))

kg = flyback.get_magcore_k_const(bmax=bsat, imax=ocp, coil_res=res, ku=ku)
print(kg)

98
1.5101246671962772e-12


In [37]:
flyback.lm_max

0.0027634874232267863